<a href="https://colab.research.google.com/github/pcpiscator/2T2021/blob/main/Experi%C3%AAncias_TCE_PB_Limpeza_e_Tratamento_da_Base%20TESTE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mpl_finance

In [2]:
# bibliotecas para análise de dados
import numpy as np
import pandas as pd
# bibliotecas para gráficos 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format
# bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.dates as mpl_dates
import sklearn
# configuração para ignorar mensagens de advertência (warning). 
import warnings
warnings.filterwarnings('ignore')


plt.style.use('ggplot')

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/pcpiscator/2T2021/main/tce_pb_L.csv', sep=';', encoding='latin_1', low_memory=False)

In [9]:
data.head(10)

,protocolo,numero_licitacao,modalidade,cd_ugestora,valor_licitado,ano_homologacao,cpf_cnpj_proponente,valor_proposta,situacao_proposta,Unnamed: 9
0,Doc. 00008/19,00002/2019,Pregao Presencial,201134,"7.320.000,00",2019,13258973000177,"7.800.000,00",Perdedora,nan
1,Doc. 00008/19,00002/2019,Pregao Presencial,201134,"500.000,00",2019,16747441000191,"7.320.000,00",Vencedora,nan
2,Doc. 00008/19,00002/2019,Pregao Presencial,201134,"7.320.000,00",2019,21969026000112,"7.800.000,00",Perdedora,nan
3,Doc. 00008/19,00002/2019,Pregao Presencial,201134,"7.320.000,00",2019,40945446000103,"7.800.000,00",Perdedora,nan
4,Doc. 00038/19,00003/2019,Pregao Presencial,201134,"1.100.000,00",2019,16747441000191,"1.680.000,00",Vencedora,nan
5,Doc. 00134/19,00001/2019,Pregao Presencial,201016,"1.072.500,00",2019,3882108000224,"55.046.500,00",Vencedora,nan
6,Doc. 00134/19,00001/2019,Pregao Presencial,201016,"2.500.000,00",2019,22410470000166,"1.110.000,00",Vencedora,nan
7,Doc. 00134/19,00001/2019,Pregao Presencial,201016,"1.680.000,00",2019,25312692000134,"2.096.500,00",Vencedora,nan
8,Doc. 00135/20,00012/2019,Inexigibilidade,201051,"81.511.840,00",2020,19710362000102,"6.500.000,00",Vencedora,nan
9,Doc. 00137/19,10001/2019,Pregao Eletronico,610095,"1.072.500,00",2019,5763509000100,"6.200.000,00",Vencedora,nan


In [6]:
#Criação de uma base com os dados originais não tratados para eventual necessidade futura
base_original = data

In [5]:
data.columns

Index(['protocolo', 'numero_licitacao', 'modalidade', 'cd_ugestora',
       'valor_licitado', 'ano_homologacao', 'cpf_cnpj_proponente',
       'valor_proposta', 'situacao_proposta', 'Unnamed: 9'],
      dtype='object')

In [8]:
#retiramos colunas desnecessárias na análise
data2 = data.drop(['Unnamed: 9'],axis=1)





In [11]:




#eliminamos todas as linhas que tenham valores nulos
data3 = data2.dropna(subset = ['protocolo', 'numero_licitacao', 'modalidade','cd_ugestora','valor_licitado','ano_homologacao', 'cpf_cnpj_proponente', 'valor_proposta', 'situacao_proposta'])

#Eliminamos todas que não forem Convite, Tomada de Preços e Dispensa (8.666)
data3['ano_homologacao'] = data3.ano_homologacao.astype(int)
data3['valor_licitado'] = data3.valor_licitado.astype(int)
data4 =data3.loc[ (data3.modalidade == 'Convite') | (data3.modalidade == 'Tomada de Preços')
| (data3.modalidade == 'Art. 24 - Lei 8.666/93') ]


#Conversão do tipo do atributo "valor_proposta" de str para int
data5['valor_proposta'] = data5['valor_proposta'].astype(str).astype(int)

#Transformo a informação da coluna "situacao_proposta" em valores de 0 e 1
data5['situacao_proposta'] = data5['situacao_proposta'].map({'Vencedora': 1, 'Perdedora': 0})

#Elimina todos os campos da variávem "cpf_cnpj_proponente" com comprimento menor que 12 ou seja, todos os CPF's
data6 = data5[data5['cpf_cnpj_proponente'].str.len() > 11]

#Renomeia a coluna "cpf_cnpj_proponente" para "cnpj_proponente"
data6.rename(columns={'cpf_cnpj_proponente': 'cnpj_proponente'}, inplace=True)

#Conversão do tipo de atributo "cnpj_proponente" de str para int
data6['cnpj_proponente'] = data6['cnpj_proponente'].astype(str).astype(int)

#Cria uma coluna chamada ident_licit, inicialmente vazia e que receberá os valores da coluna protocolo alterados e a coloca na primeira posição
data6["ident_licit"] = ""
data7=data6.reindex(columns=['ident_licit','protocolo', 'numero_licitacao', 'modalidade', 'cd_ugestora','valor_licitado', 'ano_homologacao', 'cnpj_proponente',
       'valor_proposta', 'situacao_proposta'])


ValueError: ignored

In [10]:
data7.columns

NameError: ignored

In [ ]:
#Eliminação dos caracteres especiais do atributo "protocolo"
data7['protocolo']  = data7['protocolo'].str.replace(r'\W',"")

#Eliminação das expressões "Doc. " e Prot. " do atributo "protocolo"
data7['protocolo']  = data7['protocolo'].str.replace("Doc","")
data7['protocolo']  = data7['protocolo'].str.replace("Proc","")

#A coluna "ident_licit" recebe os valores da coluna "protocolo" e esta é eliminada juntamente com a coluna "numero_licitacao"
data7['ident_licit']  = data7['protocolo']
data7 = data7.drop(['protocolo','numero_licitacao'], axis = 1)

#Conversão do tipo do atributo "idente_licit" para inteiro
data7['ident_licit'] = data7['ident_licit'].astype(str).astype(int)


In [ ]:
data7.columns

Index(['ident_licit', 'modalidade', 'cd_ugestora', 'valor_licitado',
       'ano_homologacao', 'cnpj_proponente', 'valor_proposta',
       'situacao_proposta'],
      dtype='object')

In [ ]:
data7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9509 entries, 109563 to 308687
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ident_licit        9509 non-null   int64 
 1   modalidade         9509 non-null   object
 2   cd_ugestora        9509 non-null   int64 
 3   valor_licitado     9509 non-null   int64 
 4   ano_homologacao    9509 non-null   int64 
 5   cnpj_proponente    9509 non-null   int64 
 6   valor_proposta     9509 non-null   int64 
 7   situacao_proposta  9509 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 668.6+ KB


In [ ]:
#Criação de uma Matriz de Correlação entre os atributos numéricos
correlacao = data7.corr()
correlacao.style.background_gradient(cmap='Blues')


,ident_licit,cd_ugestora,valor_licitado,ano_homologacao,cnpj_proponente,valor_proposta,situacao_proposta
ident_licit,1.000000,-0.069486,-0.266363,0.057333,0.082816,-0.250104,0.011288
cd_ugestora,-0.069486,1.000000,0.195906,-0.023885,-0.047942,0.202046,-0.092268
valor_licitado,-0.266363,0.195906,1.000000,0.021372,-0.137515,0.966956,-0.164775
ano_homologacao,0.057333,-0.023885,0.021372,1.000000,0.100667,0.021895,-0.047264
cnpj_proponente,0.082816,-0.047942,-0.137515,0.100667,1.000000,-0.136666,-0.003675
valor_proposta,-0.250104,0.202046,0.966956,0.021895,-0.136666,1.000000,-0.213280
situacao_proposta,0.011288,-0.092268,-0.164775,-0.047264,-0.003675,-0.213280,1.000000


In [ ]:
#Geração, no GDrive, de um arquivo csv com as alterações efetuadas.
data7.to_csv('/content/drive/MyDrive/00 - Pós Ciência de Dados/99 - TCC/PROJETO 00 - TCC Grupo/01 - Bases Candidatas/02 - TCE PB - Experiência com o Colab/data7.CSV', sep=',', encoding='utf-8')

## ALGUNS CÓDIGOS QUE PODEM VIR A SER ÚTEIS


In [ ]:
# mapeia cada valor da variável Embarked em um valor numérico
#mapeamento_embarked = {"S": 1, "C": 2, "Q": 3}
#treinamento_df['Embarked'] = treinamento_df['Embarked'].map(mapeamento_embarked)
#teste_df['Embarked'] = teste_df['Embarked'].map(mapeamento_embarked)